# Drive characteristics using `strym`

In this notebook, we will generate characteristics of a particular drive. Please ensure strym is installed correctly

## Installing `strym`
First, we are required to install `strym`. For that, we will just use github devtools functionality.
Even if you have installed `strym` previously, you should reinstall for latest version of the package.

In [4]:
# Installing strym using pip from git
#!pip install gmplot gmaps googlemaps dotenv
!pip install dotenv
#!pip install --upgrade --no-deps --force-reinstall git+https://github.com/jmscslgroup/strym
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/jmscslgroup/strym

  Using cached https://files.pythonhosted.org/packages/e2/46/3754073706e31670eed18bfa8a879305b56a471db15f20523c2427b10078/dotenv-0.0.5.tar.gz
    ERROR: Command errored out with exit status 1:
     command: /Users/sprinkle/opt/anaconda3/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/qb/xzd2v2fn741b459s8jvfjhk4000zl1/T/pip-install-ukkob8p5/dotenv/setup.py'"'"'; __file__='"'"'/private/var/folders/qb/xzd2v2fn741b459s8jvfjhk4000zl1/T/pip-install-ukkob8p5/dotenv/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base pip-egg-info
         cwd: /private/var/folders/qb/xzd2v2fn741b459s8jvfjhk4000zl1/T/pip-install-ukkob8p5/dotenv/
    Complete output (68 lines):
    Traceback (most recent call last):
      File "/Users/sprinkle/opt/anaconda3/lib/python3.7/site-packages/setuptools/sandbox.py", line 154, in save_modul

## Importing packages
Import required packages

In [2]:
from strym import strymread
import strym
import math
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integrate

ModuleNotFoundError: No module named 'dotenv'

## Specify Data Location

In [ ]:
# datafolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_03/"
# import glob
# csvlist = glob.glob(datafolder+"*.csv")
# datafolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020-03-05/"
# # print(csvlist)
# for f in glob.glob(datafolder+"*.csv"):
#     csvlist.append(f)
# print(csvlist)

In [ ]:
import glob
parentfolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/"
folderlist = glob.glob(parentfolder+"*")
print(folderlist)
csvlist = []
for datafolder in folderlist:
#     datafolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_03/"
    csvlisttmp = glob.glob(datafolder+"/*.csv")
    for f in csvlisttmp:
        csvlist.append(f)
#print(csvlist)

In [ ]:
num_of_files = len(csvlist)
print("Total number of datafiles in {} is {}.".format(datafolder, num_of_files))

## Analysis
### 1. CSV file containing all messages
In this section, we will extract a subset of the CSV-formatted CAN Data in order to produce summary information of a specific drive.

In [ ]:
dbcfile = '../examples/newToyotacode.dbc'
metadata = []
for i in range(num_of_files):
    try:
        r0 = strymread(csvfile=csvlist[i], dbcfile=dbcfile)
        print(f'Reading {csvlist[i]}')
        duration1 = r0.dataframe['Time'][len(r0.dataframe)-1] - r0.dataframe['Time'][0]
        duration_str = f'  Duration of this drive is {duration1} seconds ({math.trunc(int(duration1*1000)/(1000*60))} minutes {math.trunc(duration1 % 60)} seconds).'
        start_str = f'  Starting date/time of the drive is {time.ctime(r0.dataframe["Time"][0])}'

        # get the speed timeseries information from the data file
        speed_ts = r0.speed()
        # turn the timeseries into a python array for integration
        # transform from km/hr by multiplying 1000m/1km and 1 hr/3600s to get m/s
        speed_ar = np.array(speed_ts['Message'])*1000/3600
    #     speed_ar[0:-1]*1000/3600
        # find the difference of the time values
    #     dt = np.diff(np.array(speed_ts['Time']))
        dt = np.diff(speed_ts['Time'])
        # trapezoidal integration, divide by 1000 to get total km (rather than m)
        # Commented out: this produces negative values that don't make sense
    #     km_ts = np.trapz(y=speed_ar[0:-1],x=dt)/1000
        km_ts = np.trapz(y=speed_ar,x=np.array(speed_ts['Time']))/1000
        # commented out: this produces incorrect values that seem to be off by a factor of around 2, depending
        km_dx = np.trapz(y=speed_ar,dx=0.02)/1000
        # need to convert km/hr to km/s to get km later when integrating
        distance_str = f'  The trip was {km_ts} km ({km_ts*3.1/5} miles)'
    #     distance_str_dx f'  The dx version is {km_dx} km ({km_dx*3.1/5} miles)'

        #speed_ts['Time']
        #f'Total miles driven is {km_ts}'
    #     print(duration_str)
    #     print(start_str)
    #     print(distance_str)
    #     print('')
        drive = { 'filename': csvlist[i], 'distance_km': km_ts, 'distance_miles': km_ts*3.1/5, 'duration_s': duration1, 
                 'date': time.ctime(r0.dataframe["Time"][0]) }
        metadata.append( drive )
    except:
        print('Unable to process file...')
# print(metadata)

In [ ]:
# how did we do?
dist=0
for d in metadata:
    dist = dist + d['distance_miles']
print(dist)

In [ ]:
metadata